# New simulator tests and MAP

This notebook aims testing the new simulator already in the context of a fully assembled IMAGINE pipeline

In [1]:
import os, sys
sys.path.append('../')

import imagine as img
import imagine_snrs as img_snrs
import astropy.units as u
import numpy as np

In [2]:
measurements = img.observables.Measurements(img_snrs.datasets.SNR_DA530_I(),
                                            img_snrs.datasets.SNR_DA530_Q(),
                                            img_snrs.datasets.SNR_DA530_U(),
                                            img_snrs.datasets.SNR_DA530_FD())

In [3]:
L = 120*u.pc; N = 200
grid = img.fields.UniformGrid(box=[[-L,L],[-L,L],[-L,L]],
                              resolution=[N, N, N])

In [4]:
from imagine.fields import FieldFactory
from imagine.priors import FlatPrior, GaussianPrior

L_shift = 70*u.pc

te_factory = FieldFactory(grid=grid,
                          field_class=img_snrs.fields.SNRThermalElectrons,
                          active_parameters=['initial_electron_density'],
                          default_parameters={'initial_electron_density': 0.01*u.cm**-3,
                                              'shell_V0':0.0153*u.pc/u.yr, 
                                              'shell_a': 1.3, 
                                              'shell_b': 10,
                                              'elapsed_time': 1300*u.yr,
                                              'shell_radius': 35*u.pc},
                          priors={'initial_electron_density': FlatPrior(1e-4, 10, u.cm**-3),
                                  'shell_V0': FlatPrior(1e-3, 0.1, u.pc/u.yr),
                                  'shell_a': FlatPrior(0.5, 2), 
                                  'shell_b': FlatPrior(1, 50), 
                                  'elapsed_time': FlatPrior(500, 3000, u.yr),
                                  'shell_radius': FlatPrior(10, 60, u.pc)})

B_BFM_factory = FieldFactory(grid=grid,
                             field_class=img_snrs.fields.SNR_BFM_MagneticField,
                             active_parameters=['B', 'period', 'x_shift', 'y_shift'],
                             default_parameters={'B': 1*u.microgauss,
                                                 'alpha': 0*u.rad,
                                                 'beta': 0*u.rad},
                             priors={'B': FlatPrior(0, 10, u.microgauss),
                                     'x_shift': FlatPrior(-L_shift, L_shift),
                                     'y_shift': FlatPrior(-L_shift, L_shift),
                                     'period': FlatPrior(10,120, u.pc)})

CR_factory = FieldFactory(grid=grid, 
                          field_class=img_snrs.fields.EquipartitionCosmicRayElectrons,
                          active_parameters=(),
                          default_parameters={'cr_energy': 1*u.GeV, 'Ecr_Em': 1},
                          priors={'Ecr_Em': GaussianPrior(mu=1, sigma=0.1, 
                                                          xmin=1e-2, xmax=10)})


class LoSintegratorSettings(img.fields.DummyField):
    NAME = 'LoS_integrator_settings'
    FIELD_CHECKLIST = {'alpha': None,
                       'beta': None,
                       'gamma': None,
                       'distance': None}
    SIMULATOR_CONTROLLIST = {}

dummy_factory = img.fields.FieldFactory(grid=grid,
                                        field_class=LoSintegratorSettings, 
                                        active_parameters=('alpha','beta','gamma'),
                                        default_parameters={'distance': 11.3*u.kpc},
                                        priors={'alpha': img.priors.FlatPrior(-180, 180, u.deg, wrapped=True), 
                                                'beta': img.priors.FlatPrior(-90, 90, u.deg), 
                                                'gamma': img.priors.FlatPrior(-180, 180, u.deg, wrapped=True)})


In [5]:
simulator = img_snrs.simulators.LoS_integrator(measurements)

masks = img.observables.Masks()
for k in measurements:
    mask = np.isfinite(measurements[k].data)
    if np.all(mask):
        continue
    else:
        masks.append(name=k, data=mask)

likelihood = img.likelihoods.SimpleLikelihood(measurements, mask_dict=masks)



In [6]:
Pipeline = img.pipelines.UltranestPipeline

pipeline = Pipeline(run_directory=os.path.join(img.rc['temp_dir'],'uniform_field'),
                            simulator=simulator, likelihood=likelihood,
                            factory_list=[dummy_factory, B_BFM_factory, CR_factory, te_factory])


In [7]:
pipeline.test()

Sampling centres of the parameter ranges.
	Evaluating point: [0.0, 0.0, 0.0, 5.0, 65.0, 0.0, 0.0, 5.00005]


../shell/field_transformer.py:79: UserWarning: Persisting input arguments took 0.61s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.dz_dx0, self.dz_dy0, self.dz_dz0)
/home/lrodrigues/miniconda3/envs/imagine/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


	Log-likelihood -1.66562921488189e+37
	Total execution time:  29.12426319345832 s

Randomly sampling from prior.
	Evaluating point: [179.65458552 -47.50398415 -37.23093817   3.87910741  83.67206405
  60.97546991  48.48352834   3.13280384]
	Log-likelihood -9.96326954130792e+36
	Total execution time:  16.928582469001412 s

Randomly sampling from prior.
	Evaluating point: [  8.83733745 -10.17847912 -97.35220306   5.34413909 110.5358227
  -5.99132688  -9.70220059   9.39128398]
	Log-likelihood -2.4096096814669806e+37
	Total execution time:  15.976069601252675 s

Average execution time: 20.676305087904137 s


<Quantity 20.67630509 s>

In [ ]:
pipeline.get_MAP()

/home/lrodrigues/miniconda3/envs/imagine/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
